In [1]:
# Initialize the GEE API and other libraries
import ee
import geemap

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='jfsp-aspen')

In [11]:
""" Set up an interactive map """

# Load the grid (with the number of presence points in each grid as a property)
grids = ee.FeatureCollection('projects/cires-gg-earthlab/aspen-mapping/training/spatial_block_grid_50km2_count')
grids_any = grids.filter(ee.Filter.gt('n_presence',0)).size().getInfo()
print(f'Number of grids with any presence data: {grids_any}')

# Filter to only grids with >= 100 aspen presence points
grids = grids.filter(ee.Filter.gte('n_presence',100))
print(f'Number of grids with >= 100 presence points: {grids.size().getInfo()}')

# Load the Southern Rockies ecoregion and its bounds
srme = ee.FeatureCollection("EPA/Ecoregions/2013/L3").filter(ee.Filter.eq('na_l3name', 'Southern Rockies'))
bounds = srme.geometry().bounds().buffer(10000)  # apply a 10km buffer to the bounds

# Initialize a map
Map = geemap.Map()
Map.addLayerControl()

# Adding layers with styles
Map.addLayer(grids.style(**{'color': 'blue', 'fillColor': '00000000'}), {}, 'Spatial Block Grid')
Map.addLayer(srme.style(**{'color': 'green', 'fillColor': '00000000'}), {}, 'Southern Rockies')
Map.centerObject(srme, 6)
Map

Number of grids with any presence data: 95
Number of grids with >= 100 presence points: 66


Map(center=[38.74998021849737, -106.3820927255938], controls=(WidgetControl(options=['position', 'transparent_…

In [4]:
""" Prepare the image stack """

###################################
# Load and mosaic the S2 composites

# Define the folder path
folder_path = 'projects/cires-gg-earthlab/aspen-mapping/S2MSI'

# List the Image assets in the folder
asset_list = ee.data.listAssets({'parent': folder_path})['assets']
image_ids = [asset['id'] for asset in asset_list if asset['type'] == 'IMAGE']

# Create an ImageCollection from the list of images
s2 = ee.ImageCollection(image_ids).mean()

proj = s2.select('B8_summer').projection().getInfo();


###################################
# Load and mosaic the S1 composites
s1 = ee.ImageCollection(
    'projects/cires-gg-earthlab/aspen-mapping/S1SAR/S1SAR_ARD_COMPOSITES'
).mean()

# Print information about the ImageCollection
print(s1.bandNames().getInfo())


####################################
# Load elevation data from USGS 3DEP
elevation = ee.Image("USGS/3DEP/10m")

slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)

topography = elevation.addBands(slope).addBands(aspect)

print(topography.bandNames().getInfo())


#################################
# Combine image data into a stack

stack = s1.addBands(s2).addBands(topography)

print(stack.bandNames().getInfo())
print(stack.bandNames().size().getInfo())

['VV_winter', 'VH_winter', 'VV_summer', 'VH_summer', 'VV_winter_var', 'VH_winter_var', 'VV_summer_var', 'VH_summer_var', 'VV_winter_contrast', 'VH_winter_contrast', 'VV_summer_contrast', 'VH_summer_contrast', 'VV_winter_corr', 'VH_winter_corr', 'VV_summer_corr', 'VH_summer_corr', 'VV_winter_ent', 'VH_winter_ent', 'VV_summer_ent', 'VH_summer_ent']
['elevation', 'slope', 'aspect']
['VV_winter', 'VH_winter', 'VV_summer', 'VH_summer', 'VV_winter_var', 'VH_winter_var', 'VV_summer_var', 'VH_summer_var', 'VV_winter_contrast', 'VH_winter_contrast', 'VV_summer_contrast', 'VH_summer_contrast', 'VV_winter_corr', 'VH_winter_corr', 'VV_summer_corr', 'VH_summer_corr', 'VV_winter_ent', 'VH_winter_ent', 'VV_summer_ent', 'VH_summer_ent', 'B2_summer', 'B3_summer', 'B4_summer', 'B5_summer', 'B6_summer', 'B7_summer', 'B8_summer', 'B8A_summer', 'B11_summer', 'B12_summer', 'SLAVI_summer', 'CIRE_summer', 'IRECI_summer', 'MCARI_summer', 'NDVI705_summer', 'MNDWI_summer', 'B2_autumn', 'B3_autumn', 'B4_autumn', 

In [6]:
""" Add the Image Composites to the Map """

# Add the composite image to the map
# Define visualization parameters
vis_params_summer = {
    'bands': ['B4_summer', 'B3_summer', 'B2_summer'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

vis_params_autumn = {
    'bands': ['B4_autumn', 'B3_autumn', 'B2_autumn'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

Map.addLayer(stack, vis_params_summer, 'Seasonal Composite Summer')
Map.addLayer(stack, vis_params_autumn, 'Seasonal Composite Autumn')

In [13]:
""" Sample and Export the Presence Data """

sel_vars = stack.bandNames().getInfo()+['label']

# Load the presence data
Pres = ee.FeatureCollection('projects/cires-gg-earthlab/aspen-mapping/training/pi_points_srme_m500_')
Pres = Pres.select(sel_vars)
print(f'Presence samples: {Pres.size().getInfo()}')

# Sample the image stack
presSample = stack.sampleRegions(
    collection=Pres,
    properties=['label'],
    scale=10,
    tileScale=14,
    geometries=True
)

# Print the first 10 samples
print(presSample.limit(10).getInfo())

# Export to Drive
task_to_drive = ee.batch.Export.table.toDrive(
    collection=presSample,
    description="pi_points_srme_m500_sampled",
    folder="Aim1",
    fileNamePrefix="pi_points_srme_m500_sampled",
    fileFormat="GeoJSON"
)
task_to_drive.start()

# Export to Asset
task_to_asset = ee.batch.Export.table.toAsset(
    collection=presSample,
    description="pi_points_srme_m500_sampled",
    assetId="projects/cires-gg-earthlab/aspen-mapping/training/pi_points_srme_m500_sampled"
)
task_to_asset.start()

Presence samples: 9100
{'type': 'FeatureCollection', 'columns': {}, 'properties': {'band_order': ['VV_winter', 'VH_winter', 'VV_summer', 'VH_summer', 'VV_winter_var', 'VH_winter_var', 'VV_summer_var', 'VH_summer_var', 'VV_winter_contrast', 'VH_winter_contrast', 'VV_summer_contrast', 'VH_summer_contrast', 'VV_winter_corr', 'VH_winter_corr', 'VV_summer_corr', 'VH_summer_corr', 'VV_winter_ent', 'VH_winter_ent', 'VV_summer_ent', 'VH_summer_ent', 'B2_summer', 'B3_summer', 'B4_summer', 'B5_summer', 'B6_summer', 'B7_summer', 'B8_summer', 'B8A_summer', 'B11_summer', 'B12_summer', 'SLAVI_summer', 'CIRE_summer', 'IRECI_summer', 'MCARI_summer', 'NDVI705_summer', 'MNDWI_summer', 'B2_autumn', 'B3_autumn', 'B4_autumn', 'B5_autumn', 'B6_autumn', 'B7_autumn', 'B8_autumn', 'B8A_autumn', 'B11_autumn', 'B12_autumn', 'SLAVI_autumn', 'CIRE_autumn', 'IRECI_autumn', 'MCARI_autumn', 'NDVI705_autumn', 'MNDWI_autumn', 'elevation', 'slope', 'aspect']}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': Fa

In [14]:
# Check how many presence samples in blocks with >= 100 points
PresFiltered = Pres.filter(ee.Filter.bounds(grids)) # "grids" is the spatial blocks where N presence >= 100
print(f'Presence samples: {PresFiltered.size().getInfo()}')


Presence samples: 8583


In [17]:
""" Sample and Export the Background Data """

# Load the samples
Bg = ee.FeatureCollection('projects/cires-gg-earthlab/aspen-mapping/training/bg_points_evt_sbcls_')
print(f'Number of background samples: {Bg.size().getInfo()}')

# Sample the image stack
bgSample = stack.sampleRegions(
    collection=Bg,
    properties=['label'],
    scale=10,
    tileScale=14,
    geometries=True
)

# Export to Drive
task_to_drive = ee.batch.Export.table.toDrive(
    collection=bgSample,
    description="bg_points_evt_sbcls_sampled",
    folder="Aim1",
    fileNamePrefix="bg_points_evt_sbcls_sampled",
    fileFormat="GeoJSON"
)
task_to_drive.start()

# Export to Asset
asset_folder = 'projects/cires-gg-earthlab/aspen-mapping/training/'
task_to_asset = ee.batch.Export.table.toAsset(
    collection=bgSample,
    description="bg_points_evt_sbcls_sampled",
    assetId=asset_folder+"bg_points_evt_sbcls_sampled"
)
task_to_asset.start()

Number of background samples: 78891


In [12]:
# """ Sample and Export the Background Data """

# # Load the samples
# Bg = ee.FeatureCollection('projects/cires-gg-earthlab/aspen-mapping/training/bg_points_evt_sbcls')

# print(Bg.limit(10).getInfo())

# # Function to map over the feature collection
# def get_sample(ftr):
#     reduc = stack.reduceRegion(
#         reducer=ee.Reducer.mean(),
#         geometry=ftr.geometry(),
#         scale=10,
#         bestEffort=True,
#         maxPixels=1e13,
#         tileScale=16
#     )

#     # Submit the output dictionary to a feature
#     dict_to_ftr = ee.Feature(None, reduc)
#     output_ftr = ee.Feature(ftr.copyProperties(dict_to_ftr))

#     # Return the final merged table
#     return output_ftr

# # Map the function over the feature collection
# bgSample = Bg.map(get_sample)

# # Export to Drive
# task_to_drive = ee.batch.Export.table.toDrive(
#     collection=bgSample,
#     description="bg_points_evt_sbcls_sampled",
#     folder="Aim1",
#     fileNamePrefix="bg_points_evt_sbcls_sampled",
#     fileFormat="GeoJSON"
# )
# task_to_drive.start()

# # Export to Asset
# asset_folder = 'projects/cires-gg-earthlab/aspen-mapping/training/'
# task_to_asset = ee.batch.Export.table.toAsset(
#     collection=bgSample,
#     description="bg_points_evt_sbcls_sampled",
#     assetId=asset_folder+"bg_points_evt_sbcls_sampled"
# )
# task_to_asset.start()

{'type': 'FeatureCollection', 'columns': {'Block_ID': 'String', 'EVT_SBCLS': 'String', 'EVT_SBCLS_': 'Long', 'fid': 'Long', 'label': 'Float', 'system:index': 'String'}, 'version': 1705991050682443, 'id': 'projects/cires-gg-earthlab/aspen-mapping/training/bg_points_evt_sbcls', 'properties': {'system:asset_size': 3221443}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-104.7550991392277, 38.90463466611602]}, 'id': '00000000000000000000', 'properties': {'Block_ID': '-234+86', 'EVT_SBCLS': 'Annual Graminoid/Forb', 'EVT_SBCLS_': 1, 'fid': 1, 'label': 0}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-104.688021386012, 38.6282275798625]}, 'id': '00000000000000000001', 'properties': {'Block_ID': '-234+86', 'EVT_SBCLS': 'Annual Graminoid/Forb', 'EVT_SBCLS_': 1, 'fid': 2, 'label': 0}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-104.65524411367993, 38.702292396367326]}, 'id': '00000000000000000002', 'properties': {'Blo